# ETL of air pollution time series
## The deliverables
Data set containing all available recordings of hourly averaged pollutant concentrations measured in Hamburg in years 2013-2019

In [1]:
import urllib.request
import xml.etree.ElementTree as ET
from lxml import etree
import pandas as pd
import numpy as np

import re, collections
from io import StringIO
import os, fnmatch

import matplotlib.pyplot as plt

import geopandas as gpd
import mplleaflet

%matplotlib inline

In [ ]:
## Download and decompress the dataset itself:
#!mkdir Correlaid.rawData
#!mkdir Correlaid.rawData/AQD_DE_E1a_2019
#!ls -l Correlaid.rawData/
#urllib.request.urlretrieve("https://datahub.uba.de/server/rest/directories/arcgisforinspire/INSPIRE/aqd_MapServer/Daten/AQD_DE_E1a_2019.zip", "Correlaid.rawData/AQD_DE_E1a_2019.zip")
#!mv Correlaid.rawData/AQD_DE_E1a_2019.zip Correlaid.rawData/AQD_DE_E1a_2019/
#!unzip Correlaid.rawData/AQD_DE_E1a_2019/AQD_DE_E1a_2019.zip -d Correlaid.rawData/
#!rm Correlaid.rawData/AQD_DE_E1a_2019/AQD_DE_E1a_2019.zip
#!unzip Correlaid.rawData/DISKO.zip -d Correlaid.rawData/AQD_DE_E1a_2019/
#!unzip Correlaid.rawData/KONTI.zip -d Correlaid.rawData/AQD_DE_E1a_2019/
#!rm Correlaid.rawData/DISKO.zip Correlaid.rawData/KONTI.zip

#Download the rdf
#urllib.request.urlretrieve("https://www.govdata.de/ckan/dataset/luftqualitatsdaten-datenstrom-e1a-validierte-einzelwerte-2019-datensatz.rdf", "Correlaid.rawData/AQD_DE_E1a_2019/luftqualitatsdaten-datenstrom-e1a-validierte-einzelwerte-2019-datensatz.rdf")

#Download Sensor positions
#urllib.request.urlretrieve("https://datahub.uba.de/server/rest/directories/arcgisforinspire/INSPIRE/aqd_MapServer/Daten/AQD_DE_D_2019.zip", "Correlaid.rawData/AQD_DE_D_2019.zip")
#!unzip Correlaid.rawData/AQD_DE_D_2019.zip -d Correlaid.rawData/
#!rm Correlaid.rawData/AQD_DE_D_2019.zip

# Download Town-county dataset:
#urllib.request.urlretrieve("https://www.destatis.de/DE/Themen/Laender-Regionen/Regionales/Gemeindeverzeichnis/Administrativ/Archiv/GV100ADQ/GV100AD3107.zip?__blob=publicationFile",
#                           "Correlaid.rawData/GV100AD3107.zip")
#!mkdir Correlaid.rawData/GV100AD3107
#!unzip Correlaid.rawData/GV100AD3107.zip -d Correlaid.rawData/GV100AD3107/
#!rm Correlaid.rawData/GV100AD3107.zip

#!mkdir Correlaid.rawData/Geo
#urllib.request.urlretrieve("https://biogeo.ucdavis.edu/data/diva/adm/DEU_adm.zip", "Correlaid.rawData/Geo/DEU_adm.zip" 
#!unzip Correlaid.rawData/Geo/DEU_adm.zip -d Correlaid.rawData/Geo/

#!ls -la Correlaid.rawData/
#!ls -la Correlaid.rawData/AQD_DE_E1a_2019/
#!ls -la Correlaid.rawData/GV100AD3107/

#!pwd

In [2]:
# pick all tags from the XML file
Etree = ET.parse("Correlaid.rawData/AQD_DE_E1a_2019/DE_HH_2019_hour.xml")
Eroot = Etree.getroot()
Eroot.tag
Eroot.attrib
AllTags = [elem.tag for elem in Eroot.iter()]
print(AllTags[23:35])
#varName = 'observedProperty'
#print("\n".join([s for s in AllTags if varName in s]))  

['{http://www.isotc211.org/2005/gmd}result', '{http://www.isotc211.org/2005/gmd}DQ_ConformanceResult', '{http://www.isotc211.org/2005/gmd}specification', '{http://www.isotc211.org/2005/gmd}CI_Citation', '{http://www.isotc211.org/2005/gmd}title', '{http://www.isotc211.org/2005/gco}CharacterString', '{http://www.isotc211.org/2005/gmd}date', '{http://www.isotc211.org/2005/gmd}CI_Date', '{http://www.isotc211.org/2005/gmd}date', '{http://www.isotc211.org/2005/gco}Date', '{http://www.isotc211.org/2005/gmd}dateType', '{http://www.isotc211.org/2005/gmd}CI_DateTypeCode']


In [4]:
def FetchXMLentryByWord(varName, NumToPrint):
    varFull = [s for s in AllTags if varName in s][NumToPrint]
    print(varFull)
    print([(varr.attrib, varr.text) for varr in Eroot.iter(varFull)][NumToPrint])
    print('\n')
def FetchAllXMLentriesByWord(varName):
    varFull = [s for s in AllTags if varName in s][0]
    print([(varr.attrib) for varr in Eroot.iter(varFull)])
    print('\n')
def FetchAllXMLsensorID():
    varFull = [s for s in AllTags if 'value' in s][0]
    print([re.sub(r'[^a-zA-Z0-9:]*\'{http(.*)$', r'', re.sub(r'^.*AQD\/SPO.DE_', r'', str(varr.attrib))) for varr in Eroot.iter(varFull) if 'AQD' in str(varr.attrib)]) 
    print('\n')
def SelectAllXMLsensorID():
    varFull = [s for s in AllTags if 'value' in s][0]
    return([re.sub(r'[^a-zA-Z0-9:]*\'{http(.*)$', r'', re.sub(r'^.*AQD\/SPO.DE_', r'', str(varr.attrib))) for varr in Eroot.iter(varFull) if 'AQD' in str(varr.attrib)]) 


In [5]:
FetchXMLentryByWord('Quantity', 0)

FetchXMLentryByWord('uom', 2)    
FetchXMLentryByWord('observedProperty', 0)    

FetchAllXMLsensorID()

FetchXMLentryByWord('TextEncoding', 0)
ColNamesExp=SelectAllXMLsensorID()

{http://www.opengis.net/swe/2.0}Quantity
({'definition': 'http://dd.eionet.europa.eu/vocabulary/aq/primaryObservation/hour'}, '\n                ')


{http://www.opengis.net/swe/2.0}uom
({'{http://www.w3.org/1999/xlink}href': 'http://dd.eionet.europa.eu/vocabulary/uom/concentration/ug.m-3'}, None)


{http://www.opengis.net/om/2.0}observedProperty
({'{http://www.w3.org/1999/xlink}href': 'http://dd.eionet.europa.eu/vocabulary/aq/pollutant/20'}, None)


['DEHH068_CHB_dataGroup1', 'DEHH070_CHB_dataGroup1', 'DEHH008_NO2_dataGroup1', 'DEHH015_NO2_dataGroup1', 'DEHH016_NO2_dataGroup1', 'DEHH026_NO2_dataGroup1', 'DEHH033_NO2_dataGroup1', 'DEHH047_NO2_dataGroup1', 'DEHH050_NO2_dataGroup1', 'DEHH059_NO2_dataGroup1', 'DEHH064_NO2_dataGroup1', 'DEHH068_NO2_dataGroup1', 'DEHH070_NO2_dataGroup1', 'DEHH072_NO2_dataGroup1', 'DEHH073_NO2_dataGroup1', 'DEHH079_NO2_dataGroup1', 'DEHH081_NO2_dataGroup1', 'DEHH008_PM1_dataGroup1', 'DEHH015_PM1_dataGroup1', 'DEHH016_PM1_dataGroup1', 'DEHH026_PM1_dataGroup1'

In [6]:
varFull = [s for s in AllTags if 'values' in s][0]

dff=[]
for varr in Eroot.iter(varFull):
    dff.append(pd.read_csv(StringIO((varr.text).replace("@@","\n")), sep=",", header=None))

Now we have wide data frame, containing timeseries of all pollutant concentrations for all sensors. The pollutant type and the sensor ID are encoded in column names. The minimal value of pollutant concentrations -999.0 is equivalent to NA and will be imputted, as well as all negative values (the concentration can not be negative). The limit for imputation will be set to 876, i.e. NA sequences exceeding 10% of the year will not be imputted. Since the number of heavily corrupted columns is below 2%, they will be dropped in favor to the information quality:

In [68]:
dffAll=pd.concat([dff[s][4] for s in range(0,len(dff))], axis=1)
dffAll.columns=ColNamesExp


dffAll[dffAll.loc[:, dffAll.columns != 'observation_period'] < 0.0] = np.NaN # concentration cannot be negative
dffAll.interpolate(method='linear', inplace=True, axis=0, limit=876, limit_direction='both')
dffAll.insert(loc=0, column="observation_period", value=pd.to_datetime(dff[0][0]))

#dffAll.head(55)

,observation_period,DEHH068_CHB_dataGroup1,DEHH070_CHB_dataGroup1,DEHH008_NO2_dataGroup1,DEHH015_NO2_dataGroup1,DEHH016_NO2_dataGroup1,DEHH026_NO2_dataGroup1,DEHH033_NO2_dataGroup1,DEHH047_NO2_dataGroup1,DEHH050_NO2_dataGroup1,...,DEHH016_SO2_dataGroup1,DEHH059_SO2_dataGroup1,DEHH079_SO2_dataGroup1,DEHH081_SO2_dataGroup1,DEHH008_PM2_dataGroup1,DEHH015_PM2_dataGroup1,DEHH033_PM2_dataGroup1,DEHH059_PM2_dataGroup1,DEHH064_PM2_dataGroup1,DEHH068_PM2_dataGroup1
0,2019-01-01 00:00:00+01:00,2.1820,0.977000,23.896,16.787,13.292,30.217,14.883,13.441,10.037,...,2.500,2.500,2.5,2.500,98.733,116.412,51.636000,88.387,216.470,602.380
1,2019-01-01 01:00:00+01:00,0.6930,0.773000,13.698,11.791,16.222,19.486,6.349,6.496,4.000,...,2.500,2.500,2.5,9.531,33.534,96.405,75.457000,65.468,161.832,80.708
2,2019-01-01 02:00:00+01:00,0.4540,0.675000,7.991,6.998,15.669,12.586,6.243,4.708,2.000,...,2.500,2.500,2.5,11.270,24.592,25.195,15.651000,13.072,18.958,36.882
3,2019-01-01 03:00:00+01:00,0.2000,0.597667,7.322,5.273,14.999,12.025,4.714,4.130,2.000,...,2.500,2.500,2.5,2.500,22.920,16.258,11.641000,12.416,13.909,36.853
4,2019-01-01 04:00:00+01:00,0.2000,0.520333,6.211,5.665,13.821,9.234,5.180,2.000,2.000,...,5.926,2.500,2.5,2.500,30.757,19.862,15.598000,17.161,17.068,47.537
5,2019-01-01 05:00:00+01:00,0.2000,0.443000,6.546,6.040,11.198,9.614,5.901,2.000,2.000,...,2.500,2.500,2.5,2.500,27.168,20.515,20.948000,21.345,21.867,45.720
6,2019-01-01 06:00:00+01:00,0.2000,0.200000,5.902,7.353,9.200,9.744,5.551,4.170,2.000,...,2.500,2.500,2.5,2.500,10.259,12.680,10.424000,13.986,15.669,15.880
7,2019-01-01 07:00:00+01:00,0.2000,0.200000,4.849,9.311,8.340,7.092,4.932,2.000,2.000,...,2.500,2.500,2.5,2.500,3.619,6.116,1.500000,6.702,6.846,5.884
8,2019-01-01 08:00:00+01:00,0.2000,0.200000,2.000,7.082,7.081,5.452,2.000,2.000,2.000,...,2.500,2.500,2.5,2.500,6.325,1.500,3.853000,1.500,4.030,8.533
9,2019-01-01 09:00:00+01:00,0.2000,0.200000,2.000,2.000,7.151,5.465,2.000,2.000,2.000,...,2.500,2.500,2.5,2.500,8.048,1.500,5.253000,6.028,7.023,13.005


In [71]:
#dfi.isna().sum()
dffAll.drop(["observation_period"], axis=1).isna().sum().astype(bool).sum(axis=0)

4

In [72]:
print('The number of corrupted columns is ', dffAll.drop(["observation_period"], axis=1).isna().sum().astype(bool).sum(axis=0), ' of ', len(dffAll.drop(["observation_period"], axis=1).columns))

The number of corrupted columns is  4  of  79


In [73]:
dffAll.dropna(axis=1, inplace=True)

In [74]:
print('The number of corrupted columns is ', dffAll.drop(["observation_period"], axis=1).isna().sum().astype(bool).sum(axis=0), ' of ', len(dffAll.drop(["observation_period"], axis=1).columns))

The number of corrupted columns is  0  of  75


In [75]:
dffAll.head()

,observation_period,DEHH008_NO2_dataGroup1,DEHH015_NO2_dataGroup1,DEHH016_NO2_dataGroup1,DEHH026_NO2_dataGroup1,DEHH033_NO2_dataGroup1,DEHH047_NO2_dataGroup1,DEHH050_NO2_dataGroup1,DEHH059_NO2_dataGroup1,DEHH064_NO2_dataGroup1,...,DEHH016_SO2_dataGroup1,DEHH059_SO2_dataGroup1,DEHH079_SO2_dataGroup1,DEHH081_SO2_dataGroup1,DEHH008_PM2_dataGroup1,DEHH015_PM2_dataGroup1,DEHH033_PM2_dataGroup1,DEHH059_PM2_dataGroup1,DEHH064_PM2_dataGroup1,DEHH068_PM2_dataGroup1
0,2019-01-01 00:00:00+01:00,23.896,16.787,13.292,30.217,14.883,13.441,10.037,12.743,26.956,...,2.500,2.5,2.5,2.500,98.733,116.412,51.636,88.387,216.470,602.380
1,2019-01-01 01:00:00+01:00,13.698,11.791,16.222,19.486,6.349,6.496,4.000,6.534,21.644,...,2.500,2.5,2.5,9.531,33.534,96.405,75.457,65.468,161.832,80.708
2,2019-01-01 02:00:00+01:00,7.991,6.998,15.669,12.586,6.243,4.708,2.000,7.821,19.521,...,2.500,2.5,2.5,11.270,24.592,25.195,15.651,13.072,18.958,36.882
3,2019-01-01 03:00:00+01:00,7.322,5.273,14.999,12.025,4.714,4.130,2.000,4.483,11.384,...,2.500,2.5,2.5,2.500,22.920,16.258,11.641,12.416,13.909,36.853
4,2019-01-01 04:00:00+01:00,6.211,5.665,13.821,9.234,5.180,2.000,2.000,4.407,9.204,...,5.926,2.5,2.5,2.500,30.757,19.862,15.598,17.161,17.068,47.537
